In [4]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import warnings 
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('train.csv')
df = pd.get_dummies(df)


In [ ]:
text_to_find = 'noninvasive'
columns_to_drop = [col for col in df.columns if text_to_find in col]
df.drop(columns=columns_to_drop, inplace=True)

In [6]:

from sklearn.impute import KNNImputer

imr = KNNImputer(n_neighbors=10000, weights='uniform')
imr = imr.fit(df.values)
df[:]= imr.transform(df.values)

In [7]:
X = df.loc[:, df.columns != 'hospital_death']
y = df[['hospital_death']]

In [8]:
numeric_columns = df.select_dtypes(include=['int64','float64']).columns

numeric_columns = numeric_columns.drop(['RecordID', 'hospital_id', 'icu_id', 'hospital_death'])


In [9]:
from sklearn.preprocessing import RobustScaler

rbs = RobustScaler()
df[numeric_columns] = rbs.fit_transform(df[numeric_columns])

In [10]:

df_test = pd.read_csv('test.csv')
df_test = pd.get_dummies(df_test)

In [ ]:
columns_to_drop = [col for col in df_test.columns if text_to_find in col]
testdatax.drop(columns=columns_to_drop, inplace=True)

In [11]:
imr_test = KNNImputer(n_neighbors=10000, weights='uniform')
imr_test = imr_test.fit(df_test.values)
df_test[:]= imr_test.transform(df_test.values)

In [12]:
rbs_test = RobustScaler()
df_test[numeric_columns] = rbs_test.fit_transform(df_test[numeric_columns])


In [13]:
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import csv

# Set hyperparameter p=1 for Manhattan distance
scale_knn = Pipeline([("scaler", RobustScaler()), ("knr", KNeighborsClassifier(n_neighbors=1240, p=1))])
scale_knn.fit(X, y)
md_probs = scale_knn.predict_proba(df_test)
md_probs = md_probs[:, 1]
filepath = 'predictknn.csv'
with open(filepath, mode='w', newline='') as file: 
    writer = csv.writer(file)
    c = 1
    for i in md_probs:
        writer.writerow([50000 + c, i])
        c = c + 1